# Blackjack

In [435]:
import random
import numpy as np
import time
from datetime import datetime

class BlackJack:
    def __init__(self, state=None, seed=0):
        if state is None:
            self.deck = [i for i in range(52)]
            random.seed(seed)
            random.shuffle(self.deck)
            self.dealer = []
            self.player = []
            self.seed = seed
        else:
            self.set_state(state, seed=seed)

    def set_state(self, state, seed=0):
        self.deck, self.dealer, self.player = map(list, state)
        self.seed = seed
        random.seed(seed)
        random.shuffle(self.deck)

    def convert_card(self, card):
        value = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A'][card % 13]
        suit = 'CDHS'[card // 13]
        return f'{value}{suit}'

    def get_state(self):
        return self.deck, self.dealer, self.player

    def draw_player(self):
        self.player.append(self.deck.pop(0))

    def draw_dealer(self):
        self.dealer.append(self.deck.pop(0))

    def calc(self, player=True):
        hand = self.player if player else self.dealer
        has_ace, value = False, 0

        for card in hand:
            index = card % 13
            if index == 0:  # Ace
                has_ace = True
                value += 11
            elif index >= 10:  # Face cards
                value += 10
            else:
                value += index + 1

        if has_ace and value > 21:
            value -= 10

        return -1 if value > 21 else value

    def get_result(self):
        player_score, dealer_score = self.calc(True), self.calc(False)
        return np.sign(player_score - dealer_score)

    def display_game(self, move='', elapsed_time=1):
        time.sleep(elapsed_time)
        print(f'Remaining deck: {len(self.deck)}')
        if move:
            print(move)
        print(f'Dealer: {list(map(self.convert_card, self.dealer))}, Value: {self.calc(False)}')
        print(f'Player: {list(map(self.convert_card, self.player))}, Value: {self.calc(True)}\n')

def RandomPlayer(env, verbose):
    return np.random.choice(['hit', 'stick'])

def RuleBasedPlayer(env, verbose):
    threshold = 17
    return 'hit' if env.calc(True) < threshold and env.calc(True) != -1 else 'stick'

def MonteCarloPlayer(env, verbose):
    num_iter = 1000
    initial_state = env.get_state()

    stick_score = sum(Game(verbose=False, state=initial_state, seed=i, player=RandomPlayer, nextaction='stick') for i in range(num_iter))
    hit_score = sum(Game(verbose=False, state=initial_state, seed=i, player=RandomPlayer, nextaction='hit') for i in range(num_iter))

    if verbose:
        print(f"Score for stick: {stick_score}, Score for hit: {hit_score}")

    return 'stick' if stick_score > hit_score else 'hit'

def Dealer(env):
    return 'hit' if env.get_result() > 0 and env.calc(False) != -1 else 'stick'

def Game(verbose=True, state=None, seed=0, player=RandomPlayer, nextaction=None, min_sum=None):
    bj = BlackJack(state, seed)
    if verbose:
        bj.display_game()

    # Player's turn
    while bj.calc(True) != -1:
        action = nextaction or (player(bj, verbose) if min_sum is None or bj.calc(True) >= min_sum else 'hit')
        nextaction = None

        if verbose:
            print(f'Player {action.lower()}s!')
        if action == 'hit':
            bj.draw_player()
            if verbose:
                bj.display_game()
        else:
            break

    # Dealer's turn
    bj.draw_dealer()
    if verbose:
        print('Dealer shows his card.')
        bj.display_game()

    while bj.calc(False) != -1:
        action = Dealer(bj)
        if verbose:
            print(f'Dealer {action.lower()}s!')
        if action == 'hit':
            bj.draw_dealer()
            if verbose:
                bj.display_game()
        else:
            break

    result = bj.get_result()
    if verbose:
        print('Draw' if result == 0 else 'Player wins' if result == 1 else 'Dealer wins')
    return result

def GenerateStart(seed=None):
    seed = seed or datetime.now().microsecond
    env = BlackJack(seed=seed)
    env.draw_player()
    env.draw_player()
    env.draw_dealer()
    return env.get_state()


In [436]:
# Simulating games
num_games = 100

# Random Agent
random_score = sum(Game(verbose=False, state=GenerateStart(i), seed=11, player=RandomPlayer) for i in range(num_games))
print(f"Overall score of Random Agent: {random_score} over {num_games} games")

# Rule-Based Agent
rule_based_score = sum(Game(verbose=False, state=GenerateStart(i), seed=11, player=RuleBasedPlayer) for i in range(num_games))
print(f"Overall score of Rule-Based Agent: {rule_based_score} over {num_games} games")

# Monte Carlo Agent
monte_carlo_score = sum(Game(verbose=False, state=GenerateStart(i), seed=11, player=MonteCarloPlayer) for i in range(num_games))
print(f"Overall score of Monte Carlo Agent: {monte_carlo_score} over {num_games} games")

Overall score of Random Agent: -48 over 100 games
Overall score of Rule-Based Agent: -19 over 100 games
Overall score of Monte Carlo Agent: -17 over 100 games
